In [99]:
import numpy as np

# Question 3 

In [100]:
n=int(input("Number of links: "))
dh=[0]*n
R_or_P=[0]*n
# print("type 0 for Revolute joint and 1 for prismatic joint")
print("Link | theta_i | d_i | a_i | alpha_i")
for i in range(n):
    dh[i]=[eval(j) for j in input(str(i+1)+" "*6).split()]
lst=[int(i) for i in input("type the joint numbers of prismatic joints (leave empty if none): ").split()]
for i in lst:
    R_or_P[i-1]=1
# print(dh)

Number of links: 3
Link | theta_i | d_i | a_i | alpha_i
1      0 1 1 0
2      0 0 1 0
3      0 -0.1 0 0
type the joint numbers of prismatic joints (leave empty if none): 3


In [101]:
A=[0]*n
for i in range(n):
    theta, d, a, alpha = dh[i]
    A[i]=np.array([[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
                   [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
                   [0, np.sin(alpha), np.cos(alpha), d],
                   [0, 0, 0, 1]])

In [102]:
ans=np.identity(4)
Z=[0]*(n+1)
Z[0]=np.array([[0],
               [0],
               [1]])

O=[0]*(n+1)
O[0]=np.array([0, 0, 0])

for i in range(n):
    ans=ans@A[i]
    Z[i+1]=ans[0:3, 0:3]@Z[0]
    O[i+1]=ans[0:3,3].T
#     O[i+1]=np.array([ans[0:3,3]]).T
print("position of end effector:", ans[0:3,3])
print("orientation of end effector:")
print(ans[0:3,0:3])
# print(np.around(ans,2))

position of end effector: [2.  0.  0.9]
orientation of end effector:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


jakobian

In [103]:
J=[0]*n
for i in range(n):
    if R_or_P[i]==1:
        J[i]=np.concatenate((Z[i],np.array([[0,0,0]]).T),axis=0)
    else:
        cross=np.cross(Z[i].T,(O[n]-O[i]).T)
        J[i]=np.concatenate((cross.T,Z[i]),axis=0)

final_J=J[0]
for i in range(1,n):
    final_J=np.concatenate((final_J,J[i]),axis=1)

print(final_J)
# print(np.around(final_J,2))
        

[[ 0. -0.  0.]
 [ 2.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]]


In [109]:
dots=input("endter q_dot for revolute joint and d_dot for prismatic joint: ("+str(n)+" values) ")
print()
q_dots=[int(i) for i in dots.split()]
print("End-effector velocity:")
print((final_J@q_dots)[0:3])

endter q_dot for revolute joint and d_dot for prismatic joint: (3 values) 1 1 1

End-effector velocity:
[0. 3. 1.]


# Question 11

In [206]:
import sympy as sym

In [207]:
n=2
q1 = sym.Symbol('q1')
q1_dot = sym.Symbol('q1_dot')
q1_dot_dot = sym.Symbol('q1_dot_dot')
q2 = sym.Symbol('q2')
q2_dot = sym.Symbol('q2_dot')
q2_dot_dot = sym.Symbol('q2_dot_dot')
l1=1
l2=1
m1=1
m2=1
g=9.81
D=np.array([[m1*l2**2/3+m2*l2**2, m2*l1*l2/2*sym.cos(q2-q1)],
             [m2*l1*l2/2*sym.cos(q2-q1), m2*l2**2/3]])
V=m1*g*l1/2*sym.sin(q1)+m2*g*(l1*sym.sin(q1)+l2/2*sym.sin(q2))
phi=np.array([[sym.diff(V, q1)],
              [sym.diff(V, q2)]])
q=np.array([[q1],
            [q2]])
q_dot=np.array([[q1_dot],
                [q2_dot]])
q_dot_dot=np.array([[q1_dot_dot],
                    [q2_dot_dot]])

In [208]:
c=[0]*n
for k in range(n):
    for i in range(n):
        for j in range(n):
            sum=sym.diff(D[k][j], "q"+str(i))+sym.diff(D[k][i], "q"+str(j))+sym.diff(D[i][j], "q"+str(k))
            c[k]+=0.5*(sum)*sym.Symbol("q"+str(i))*sym.Symbol("q"+str(j))

In [209]:
final=D@q_dot_dot+phi+np.transpose([c])
print(final)

[[-0.5*q1**2*sin(q1 - q2) + 1.33333333333333*q1_dot_dot + 0.5*q2_dot_dot*cos(q1 - q2) + 14.715*cos(q1)]
 [-1.0*q0*q1*sin(q1 - q2) + 0.5*q1_dot_dot*cos(q1 - q2) + 0.333333333333333*q2_dot_dot + 4.905*cos(q2)]]


The above printed matrix is left side of the equation written in question 10. which equals to the torque matrix.